**Copernicus Climate Datastore **

Copernicus Programme is the remote sensing program organized by the European Union and ESA (European Space Agency). Besides Sentinel-2 that you can access through Google Earth Engine, another crucial product is ERA5 and ERA5-Land.

ERA5-Land is a climate reanalysis dataset containing a significant number of climate parameters (temperature, humidity, wind speed, snowmelt, etc) on a global scale in the period from 1950 to 2024 with regular updates.

In this notebook, you will learn a general way to obtain ERA5-Land data by using CDS API.

Note: for this notebook, as with any API, you will need to obtain your ID and key from website by registering. You can use the following link for the main hub: https://cds.climate.copernicus.eu/ . Main manual for API access can be obtained here: https://cds.climate.copernicus.eu/how-to-api  .



# Setup Environment

In [ ]:
import os

url = 'url: https://cds.climate.copernicus.eu/api' # keep it as is
key = 'key: ***KEY***' # change ***key*** to API pi from your Login page

# Get the home directory
home_dir = os.path.expanduser("~")
file_path = os.path.join(home_dir, '.cdsapirc')

# Write the configuration to the file
with open(file_path, 'w') as f:
    f.write('\n'.join([url, key]))

# Read and print the configuration from the file
with open(file_path) as f:
    print(f.read())

url: https://cds.climate.copernicus.eu/api
key: ***KEY***


In [ ]:
!pip install cdsapi

  Preparing metadata (setup.py) ... done
  Created wheel for multiurl: filename=multiurl-0.3.3-py3-none-any.whl size=21230 sha256=5701276d25a05eaad2528aee5d6c899653a7f0ea9f2f88863328b664048636e2
  Stored in directory: /root/.cache/pip/wheels/be/05/e0/65a6edb0a000498aeaefbadd80228bf5ed1bdbb82840ca1692
Successfully built multiurl


In [ ]:
!pip install netCDF4 ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.9 MB/s eta 0:00:00


# Download Dataset using API

There are several parameters that you should define before proceeding with your data download.
1. Product - this is basically the name of a dataset. You can check the official Copernicus website for any other datasets. In our case, monthly means is defined with **'reanalysis-era5-land-monthly-means'**
2. Variable - this is basically what variables you wish to choose. You can find the list of possible variables here: https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-monthly-means?tab=overview . Also, I recommend you to read the reference guide as it provides useful background on what data represents and how it was obtained in the first place.
3. Month - this is a selection of months in your data file. If your study necessitates only seasonal data, all you need to do is just list necessary months
4. Format - this is a data file format. If you want to continue working in Python directly, netcdf is by far the easiest data format to operate in.
5. Time - for monthly data, it is defaulted to '00:00' so no need to modify it. If you chose hourly data, that's where your hours selection goes.
6. Year - selection of years of data. Dataset is limited to 1950 to 2024 (only **1950~2023 **has all months available)
7. Product Type - it can be monthly means (which averages data over the whole month of the year, useful for long-term changes), or hourly means (which averages data of the hourly scale, useful for diurnal variation). In our case, monthly means is defined with **'monthly_averaged_reanalysis'**

This function is already predefined to download data for every year of the months in a period [start_year, end_year]

Keep in mind - there's a lot of researchers worldwide using Copernicus and a limited number of slots. This means that any requests you may put will be placed into a live queue. In non-peak hours (night in Europe/USA) you may be able to download almost immediately, while in peak hours you may have to wait a couple of hours for your turn. Luckily, even if your Colab has timed out, you can always access your already requested data directly from Copernicus Website.

In [ ]:
import cdsapi

def download_era5_land_data(start_year, end_year):
    c = cdsapi.Client()

    dataset = "reanalysis-era5-land-monthly-means"

    for year in range(start_year, end_year + 1):
        print(f"Downloading data for year {year}")

        request = {
            "product_type": ["monthly_averaged_reanalysis"],
            "variable": [
                "2m_dewpoint_temperature",
                "2m_temperature",
                "skin_temperature",
                "snow_cover",
                "10m_u_component_of_wind",
                "10m_v_component_of_wind"
            ],
            "year": [f"{year}"],
            "month": [
                "01", "02", "03",
                "04", "05", "06",
                "07", "08", "09",
                "10", "11", "12"
            ],
            "time": ["00:00"],
            "data_format": "netcdf",
            "download_format": "unarchived"
        }

        # File path for saving the downloaded data
        output_file = f"dataset/download_{year}.nc"

        # Download the data
        c.retrieve(dataset, request).download(output_file)

        print(f"Download complete for year {year}")

# Example usage
start_year = 2023
end_year = 2023

download_era5_land_data(start_year, end_year)

2024-12-15 20:48:14,608 INFO [2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
INFO:datapi.legacy_api_client:[2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
2024-12-15 20:48:14,612 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
INFO:datapi.le

2024-12-15 20:48:15,061 INFO Request ID is 1befcd26-73df-4863-8451-cf804c9093cf
INFO:datapi.legacy_api_client:Request ID is 1befcd26-73df-4863-8451-cf804c9093cf
2024-12-15 20:48:15,226 INFO status has been updated to accepted
INFO:datapi.legacy_api_client:status has been updated to accepted
2024-12-15 21:18:55,814 INFO status has been updated to successful
INFO:datapi.legacy_api_client:status has been updated to successful


6d5422a01627015de0a6b6a89f8f5c09.nc:   0%|          | 0.00/231M [00:00<?, ?B/s]

Download complete for year 2023


At this point, you are pretty much done! Your data is succesfully saved in the 'dataset/' folder with each year having separate files of identical structure.

Now, as an example, the files here contain following variables: u- and v-components of wind speed, air temperature at 2m above ground, skin temperature (temperature right between ground and air) and snowcover. Let's see how we can utilize them.

In [ ]:
import matplotlib.pyplot as plt
import netCDF4
import numpy as np
from ipywidgets import interactive
from IPython.display import display
from mpl_toolkits.axes_grid1 import make_axes_locatable

def plot_netcdf_variables(nc):
    variables = ['u10', 'v10', 'd2m', 't2m', 'skt']

    def update_plots(month_index=0):
        fig, axs = plt.subplots(3, 2, figsize=(15, 20))
        fig.suptitle(f'Weather Variables for Month {month_index + 1}', fontsize=16)

        for i, var_name in enumerate(variables):
            if var_name not in nc.variables:
                print(f"Warning: Variable '{var_name}' not found in the NetCDF file.")
                continue

            var = nc[var_name]
            if len(var.shape) == 3:  # Assuming shape is (time, lat, lon)
                data = var[month_index, :, :]
            else:
                print(f"Error: Unexpected shape {var.shape} for variable '{var_name}'.")
                continue

            row = i // 2
            col = i % 2

            if var_name == 'skt':
                ax = axs[2, 0]
                fig.delaxes(axs[2, 1])  # Remove the unused subplot
                ax.set_position([0.1, 0.1, 0.8, 0.25])  # Adjust this to change the size of the skt plot
            else:
                ax = axs[row, col]

            im = ax.imshow(data, cmap='viridis')
            ax.set_title(f'{var_name}')

            # Create a new axes for the colorbar with the same height as the plot
            divider = make_axes_locatable(ax)
            cax = divider.append_axes("right", size="5%", pad=0.01)

            plt.colorbar(im, cax=cax, label=f"{var.units if hasattr(var, 'units') else ''}")
            ax.set_xlabel("Longitude")
            ax.set_ylabel("Latitude")

        plt.tight_layout()
        plt.subplots_adjust(hspace=0.1, wspace=0.1)  # Adjust spacing between subplots
        plt.show()

    month = 6  # Default month index
    interactive_plot = interactive(update_plots, month_index = month)  # Assuming 12 months of data
    display(interactive_plot)

# Main execution
fp = 'dataset/download_2023.nc'
nc = netCDF4.Dataset(fp)

# Generate plots
plot_netcdf_variables(nc)

# Note: The NetCDF file is kept open for interactive plotting.
# You may want to add a cell to close it when you're done:
# nc.close()

interactive(children=(IntSlider(value=6, description='month_index', max=18, min=-6), Output()), _dom_classes=(…

Sometimes having raw data as is might not be too helpful. For example, we can't directly use wind speed or dewpoint temperature, but we can use relative humidity and wind speed-direction. Thus, we can apply formulas over entire layers in NetCDF file to create new layers of data.

In [ ]:
def plot_wind_speed_direction(nc):
    def update_wind_plots(month_index=0):
        u10 = nc['u10'][month_index, :, :]
        v10 = nc['v10'][month_index, :, :]

        V = np.sqrt(u10**2 + v10**2)
        Phi = np.mod(180 + 180/np.pi * np.arctan2(v10, u10), 360)

        fig, axs = plt.subplots(1, 2, figsize=(20, 10))
        fig.suptitle(f'Wind Variables for Month {month_index + 1}', fontsize=16)

        # Plot V (wind speed)
        ax_V = axs[0]
        im_V = ax_V.imshow(V, cmap='viridis')
        ax_V.set_title('V (Wind Speed)')
        divider_V = make_axes_locatable(ax_V)
        cax_V = divider_V.append_axes("right", size="5%", pad=0.05)
        plt.colorbar(im_V, cax=cax_V, label='m/s')
        ax_V.set_xlabel("Longitude")
        ax_V.set_ylabel("Latitude")

        # Plot Phi (wind direction)
        ax_Phi = axs[1]
        im_Phi = ax_Phi.imshow(Phi, cmap='viridis')
        ax_Phi.set_title('Phi (Wind Direction)')
        divider_Phi = make_axes_locatable(ax_Phi)
        cax_Phi = divider_Phi.append_axes("right", size="5%", pad=0.05)
        plt.colorbar(im_Phi, cax=cax_Phi, label='degrees')
        ax_Phi.set_xlabel("Longitude")
        ax_Phi.set_ylabel("Latitude")

        plt.tight_layout()
        plt.show()

    month = 0  # Default month index
    interactive_wind_plot = interactive(update_wind_plots, month_index=(0, 11))  # Assuming 12 months of data
    display(interactive_wind_plot)

# Call the new function to generate plots for wind speed and direction
plot_wind_speed_direction(nc)

interactive(children=(IntSlider(value=0, description='month_index', max=11), Output()), _dom_classes=('widget-…

In [ ]:
def plot_relative_humidity(nc):
    def update_rh_plot(month_index=6):
        d2m = nc['d2m'][month_index, :, :] - 273.15
        t2m = nc['t2m'][month_index, :, :] - 273.15

        A = np.exp((17.625 * d2m) / (243.04 + d2m))
        B = np.exp((17.625 * t2m) / (243.04 + t2m))

        RH = 100 * (A / B)

        fig, ax = plt.subplots(figsize=(10, 8))
        im = ax.imshow(RH, cmap='viridis')
        ax.set_title('Relative Humidity (RH)')
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.05)
        plt.colorbar(im, cax=cax, label='%')
        ax.set_xlabel("Longitude")
        ax.set_ylabel("Latitude")

        plt.tight_layout()
        plt.show()

    month = 0  # Default month index
    interactive_rh_plot = interactive(update_rh_plot, month_index=(0, 11))  # Assuming 12 months of data
    display(interactive_rh_plot)

# Call the new function to generate plot for relative humidity
plot_relative_humidity(nc)

interactive(children=(IntSlider(value=6, description='month_index', max=11), Output()), _dom_classes=('widget-…

Now, sometimes, you may have chosen a topic of study, but still struggle with finding the right location to assess. It might be useful to analyze the parameters you are working with and see whether some areas on Earth have more favourable combination of parameters. For example, factors for reduced snow melt could include high enough wind speed, low enough relative humidity, and sub-zero temperatures. So, we could check out what regions of the planet have optimal conditions to facilitate it and plot them.

In [ ]:
# Additional Code Block for Plotting Conditions

import netCDF4
import requests

def download_file(url, dest_path):
    response = requests.get(url)
    with open(dest_path, 'wb') as file:
        file.write(response.content)

def plot_conditions(nc):
    # Download the glacier data from the provided NetCDF file
    glacier_fp = '/content/cicecap_v015_1279_4_regMIR.nc'
    download_url = 'https://confluence.ecmwf.int/download/attachments/140385202/cicecap_v015_1279_4_regMIR.nc?version=1&modificationDate=1711562368310&api=v2'
    download_file(download_url, glacier_fp)

    glacier_nc = netCDF4.Dataset(glacier_fp)

    def update_conditions_plot(month_index=6):
        u10 = nc['u10'][month_index, :, :]
        v10 = nc['v10'][month_index, :, :]
        skt = nc['skt'][month_index, :, :] - 273.15
        d2m = nc['d2m'][month_index, :, :] - 273.15
        t2m = nc['t2m'][month_index, :, :] - 273.15
        snow_cover = nc['snowc'][month_index, :, :]

        # Calculate wind speed
        V = np.sqrt(u10**2 + v10**2)

        # Calculate relative humidity
        A = np.exp((17.625 * d2m) / (243.04 + d2m))
        B = np.exp((17.625 * t2m) / (243.04 + t2m))
        RH = 100 * (A / B)

        # Initialize the condition satisfaction matrix
        conditions = np.zeros(skt.shape)

        # High wind speed condition, in m/s
        conditions += (V > 3).astype(int)

        # Low temperature condition, in C
        conditions += (skt <= 0).astype(int)

        # Low relative humidity condition, in %
        conditions += (RH < 60).astype(int)

        # Glacier condition
        conditions += (snow_cover >= 0.5).astype(int)

        # Latitude condition (above 63 degrees south)
        #latitudes = np.linspace(90, -90, skt.shape[0])
        #latitude_condition = np.tile(latitudes[:, np.newaxis], (1, skt.shape[1]))
        #conditions += (latitude_condition >= -63).astype(int)

        # Count the number of perfect fit tiles
        perfect_fit = np.sum(conditions == 4)


        # Plotting
        fig, axs = plt.subplots(1, 2, figsize=(20, 8))
        plt.title(label= "Parameters fit during Month " + str(month_index) + " . Perfect Fit Tiles: " + str(perfect_fit), loc = 'left', fontsize = 16, y = -0.17)
        # Plot skin temperature
        im1 = axs[0].imshow(skt, cmap='coolwarm')
        axs[0].set_title('Skin Temperature (skt)')
        divider1 = make_axes_locatable(axs[0])
        cax1 = divider1.append_axes("right", size="5%", pad=0.05)
        plt.colorbar(im1, cax=cax1, label=f"{nc['skt'].units}")

        # Plot condition satisfaction
        im2 = axs[1].imshow(conditions, cmap='RdYlGn')
        axs[1].set_title('Condition Satisfaction')
        divider2 = make_axes_locatable(axs[1])
        cax2 = divider2.append_axes("right", size="5%", pad=0.05)
        plt.colorbar(im2, cax=cax2, label='Conditions Matched (0 to 4)')

        for ax in axs:
            ax.set_xlabel("Longitude")
            ax.set_ylabel("Latitude")
        plt.tight_layout()
        plt.show()

    interactive_conditions_plot = interactive(update_conditions_plot, month_index=(0, 11))  # Assuming 12 months of data
    display(interactive_conditions_plot)

# Call the new function to generate plots for conditions
plot_conditions(nc)

interactive(children=(IntSlider(value=6, description='month_index', max=11), Output()), _dom_classes=('widget-…